In [1]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import pandas as pd
import numpy as np
import time
import sklearn
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import pylab as pl
import seaborn as sns
import kmedoids

In [2]:
df_log = pd.read_csv("C:/Users/djcsr/Documents/Semester 9/CGS4144 Bioinformatics/Analysis/data/SRP073813/SRP073813_log.csv")
df_meta = pd.read_csv("C:/Users/djcsr/Documents/Semester 9/CGS4144 Bioinformatics/Analysis/data/SRP073813/SRP073813_meta.csv")

In [3]:
df_log.set_index("HUGO", inplace = True)
df_meta.set_index("ID", inplace = True) #Setting the index for data makes the enrichment analysis easier
df_meta = df_meta['refinebio_subject'] #removes unnecessary data 

In [4]:
df_meta = df_meta.to_frame()

In [5]:
df_log =round(df_log) 
df_log = df_log.T #swaps columns and indexes
df_log = df_log.join(df_meta)

In [6]:
df_log = df_log.loc[:, ~df_log.columns.duplicated()] #removes duplicate columns prevent deseq

In [7]:
#Transforming the data to contain only data collected from a certain part of the brain in our patients 
df_log = df_log[df_log["refinebio_subject"].str.contains('nacc')]
df_meta = df_meta[df_meta["refinebio_subject"].str.contains('nacc')] 

In [8]:
df_log.drop("refinebio_subject", axis = 1, inplace = True)

In [9]:
df_log = df_log.T

In [10]:
df_log

,SRR3438559,SRR3438560,SRR3438561,SRR3438562,SRR3438615,SRR3438616,SRR3438617,SRR3438618,SRR3438619,SRR3438620,...,SRR3438856,SRR3438863,SRR3438869,SRR3438891,SRR3438897,SRR3438899,SRR3438902,SRR3438903,SRR3438904,SRR3438905
TSPAN6,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
TNMD,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
DPM1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
SCYL3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
FIRRM,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NOTCH2NLC,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0
ARMCX5-GPRASP2,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
LOC112268419,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
LOC124903324,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [11]:
df_log["variance"] = np.var(df_log.values, axis=1)
df = df_log.sort_values("variance", ascending=False)
df.drop("variance", axis =1, inplace = True)
dfTen = df.head(10)
dfHun = df.head(100)
dfThous = df.head(1000)
dfTenThous = df.head(10000)
dfFiveThous = df.head(5000)

In [12]:
dfTen = dfTen.T
dfHun = dfHun.T
dfThous = dfThous.T
dfTenThous = dfTenThous.T
dfFiveThous = dfFiveThous.T

In [13]:
dfFiveThous

,CHI3L1,PENK,PTPN5,SYNDIG1L,XIST,HSPA1A,RAP1GAP,PEG3,MOBP,NRGN,...,KLHDC7B,ADAM20,SCFD1,MMP28,VLDLR,ENAM,LINC01980,ICE2,ZXDC,LL22NC03-63E9.3
SRR3438559,2.0,5.0,6.0,7.0,4.0,5.0,6.0,5.0,7.0,6.0,...,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0
SRR3438560,3.0,6.0,7.0,7.0,2.0,5.0,6.0,5.0,5.0,7.0,...,1.0,1.0,2.0,2.0,3.0,2.0,1.0,2.0,3.0,2.0
SRR3438561,3.0,6.0,7.0,7.0,2.0,4.0,7.0,5.0,5.0,6.0,...,1.0,1.0,2.0,3.0,2.0,2.0,1.0,3.0,3.0,1.0
SRR3438562,3.0,6.0,7.0,7.0,4.0,4.0,7.0,6.0,5.0,6.0,...,2.0,1.0,3.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0
SRR3438615,3.0,6.0,7.0,7.0,2.0,5.0,6.0,5.0,4.0,7.0,...,2.0,1.0,2.0,3.0,3.0,1.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR3438899,2.0,5.0,6.0,7.0,2.0,4.0,6.0,6.0,7.0,7.0,...,1.0,2.0,3.0,2.0,3.0,2.0,1.0,3.0,2.0,2.0
SRR3438902,3.0,2.0,3.0,3.0,2.0,7.0,3.0,4.0,7.0,3.0,...,1.0,1.0,2.0,3.0,3.0,1.0,1.0,3.0,3.0,2.0
SRR3438903,3.0,3.0,5.0,6.0,2.0,4.0,6.0,6.0,6.0,6.0,...,1.0,2.0,3.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0
SRR3438904,5.0,3.0,5.0,4.0,2.0,7.0,5.0,4.0,7.0,5.0,...,1.0,1.0,3.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0


In [14]:
# Above is code needed until we diverge on clustering method

In [15]:
dfTen.shape

(114, 10)

In [16]:
dissDfTen = euclidean_distances(dfTen)
dissDfHun = euclidean_distances(dfHun)
dissDfThous = euclidean_distances(dfThous)
dissDfTenThous = euclidean_distances(dfTenThous)
dissDfFiveThous = euclidean_distances(dfFiveThous)

In [17]:
# clustering using PAM on ten genes
tenPamk5 = kmedoids.pam(dissDfTen, medoids = 5, max_iter = 100, init = "first", random_state = None) 
tenPamk4 = kmedoids.pam(dissDfTen, medoids = 4, max_iter = 100, init = "first", random_state = None) 
tenPamk3 = kmedoids.pam(dissDfTen, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [18]:
tenPamk5, tenPamk4, tenPamk3

(KMedoidsResult(loss=254.01457450512822, labels=[0 1 1 3 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 3 3 0 0 2 2 3
  3 1 1 0 0 1 3 3 0 0 1 3 3 2 4 2 1 4 1 0 0 0 4 0 0 3 3 0 2 1 3 0 0 0 0 1 1
  2 3 1 3 3 0 0 3 0 0 1 1 1 1 1 0 1 0 1 3 0 2 1 1 2 2 0 1 3 0 0 0 2 0 1 0 4
  0 2 2], medoids=[ 60  47 113   3  51], n_iter=5, n_swaps=4),
 KMedoidsResult(loss=271.39245512067566, labels=[0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 3 3 3
  1 1 1 0 0 1 1 1 0 0 1 0 3 3 2 3 1 2 1 0 0 0 2 0 0 0 0 0 3 1 1 0 0 0 0 1 1
  3 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 0 3 1 1 3 3 0 1 0 0 0 0 3 0 1 0 2
  0 3 3], medoids=[ 60  47  51 113], n_iter=5, n_swaps=4),
 KMedoidsResult(loss=291.1972636074242, labels=[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1
  1 1 1 0 0 1 1 1 0 0 1 0 1 0 2 0 1 2 1 0 0 0 2 0 0 0 0 0 0 1 1 0 1 0 0 1 1
  0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 0 0 0 1 0 1 0 2
  0 0 0], medoids=[60 47 54], n_iter=4, n_swaps=3))

In [19]:
#clustering using PAM on one hundred genes
hunPamk5 = kmedoids.pam(dissDfHun, medoids = 5, max_iter = 100, init = "first", random_state = None) 
hunPamk4 = kmedoids.pam(dissDfHun, medoids = 4, max_iter = 100, init = "first", random_state = None) 
hunPamk3 = kmedoids.pam(dissDfHun, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [20]:
hunPamk5, hunPamk4, hunPamk3

(KMedoidsResult(loss=760.9986735816505, labels=[1 1 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 4 4 4 4 4 4 4
  4 4 4 4 4 3 3 3 0 1 3 3 3 0 2 0 3 2 3 1 0 1 2 1 1 3 3 1 0 3 3 0 1 1 0 3 3
  3 3 3 3 3 1 1 3 1 1 3 3 3 1 3 1 3 3 3 3 1 0 1 3 3 3 1 3 3 1 1 1 1 0 3 1 0
  0 0 1], medoids=[95 83 54 77 24], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=804.5763558200044, labels=[0 0 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1
  1 1 1 0 0 3 3 3 0 0 3 3 3 0 2 0 3 2 3 0 0 0 2 0 0 3 3 0 0 3 3 0 0 0 0 3 3
  3 3 3 3 3 0 0 3 3 0 0 3 3 3 3 0 3 3 3 3 0 0 3 3 3 3 0 3 3 0 0 0 0 0 3 0 0
  0 0 0], medoids=[103  24  54  77], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=874.13285805174, labels=[1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2
  2 2 2 2 2 1 2 2 1 1 2 1 2 1 0 1 2 0 2 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2
  1 2 1 1 2 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1
  1 1 1], medoids=[54 83 25], n_iter=5, n_swaps=4))

In [21]:
#clustering using PAM on one thousand genes
thousPamk5 = kmedoids.pam(dissDfThous, medoids = 5, max_iter = 100, init = "first", random_state = None) 
thousPamk4 = kmedoids.pam(dissDfThous, medoids = 4, max_iter = 100, init = "first", random_state = None) 
thousPamk3 = kmedoids.pam(dissDfThous, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [22]:
thousPamk5, thousPamk4, thousPamk3

(KMedoidsResult(loss=2132.404262784683, labels=[1 1 4 4 4 4 1 1 4 4 4 4 4 4 4 4 4 4 1 1 4 4 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
  1 4 4 1 1 3 4 4 0 1 3 3 3 0 2 0 3 2 3 0 0 3 2 3 3 3 3 3 0 3 3 0 3 1 0 3 3
  3 3 3 3 3 0 0 3 3 3 3 3 3 3 3 1 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 0 3 3 0
  0 0 3], medoids=[ 71  33  54 101  12], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=2198.6550842318156, labels=[0 0 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1
  1 1 1 1 1 3 3 3 0 0 3 3 3 0 2 0 3 2 3 0 0 0 2 0 0 3 3 0 0 3 3 0 0 0 0 3 3
  3 3 3 3 3 0 0 0 0 0 3 3 3 3 3 0 3 3 3 3 0 0 0 3 3 3 0 3 3 0 0 0 0 0 3 0 0
  0 0 0], medoids=[83 24 54 77], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=2332.8742495531055, labels=[1 1 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 1 1 1 1 2 2 2 2 2
  2 2 2 1 1 1 2 2 1 1 1 1 2 1 0 1 2 0 2 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2
  1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1
  1 1 1], medoids=[54 83 12], n_iter=4, n_swaps=3))

In [23]:
#clustering using PAM on ten thousand genes
tenThousPamk5 = kmedoids.pam(dissDfTenThous, medoids = 5, max_iter = 100, init = "first", random_state = None) 
tenThousPamk4 = kmedoids.pam(dissDfTenThous, medoids = 4, max_iter = 100, init = "first", random_state = None) 
tenThousPamk3 = kmedoids.pam(dissDfTenThous, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [24]:
tenThousPamk5, tenThousPamk4, tenThousPamk3

(KMedoidsResult(loss=5760.533354616908, labels=[1 1 4 4 4 4 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 4 4 4 4 4 4 4
  4 4 4 4 4 3 3 3 0 1 3 3 3 0 2 0 3 2 3 1 1 1 2 1 3 3 3 1 0 3 3 1 1 1 0 3 3
  3 3 3 3 3 0 0 1 1 1 3 3 3 3 3 1 3 3 3 3 1 0 0 3 3 1 1 3 3 1 1 0 0 0 3 3 0
  0 0 1], medoids=[71 83 51 77 12], n_iter=7, n_swaps=6),
 KMedoidsResult(loss=5908.939623649717, labels=[1 1 3 3 3 3 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 3 3 3 3 1 1 3 3 3 3 3 3 3
  3 3 3 3 3 1 1 1 2 1 1 1 1 2 0 2 1 0 1 1 2 1 0 1 1 1 1 1 2 1 1 1 1 1 2 1 1
  1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 2 1 1 2
  2 2 1], medoids=[51 82 71 12], n_iter=5, n_swaps=4),
 KMedoidsResult(loss=6164.435096199542, labels=[0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
  1 1 1 1 1 0 0 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0], medoids=[62 12 51], n_iter=5, n_swaps=4))

In [25]:
#clustering using PAM on five thousand genes
fiveThousPamk5 = kmedoids.pam(dissDfFiveThous, medoids = 5, max_iter = 100, init = "first", random_state = None) 
fiveThousPamk4 = kmedoids.pam(dissDfFiveThous, medoids = 4, max_iter = 100, init = "first", random_state = None) 
fiveThousPamk3 = kmedoids.pam(dissDfFiveThous, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [26]:
fiveThousPamk5, fiveThousPamk4, fiveThousPamk3

(KMedoidsResult(loss=4504.889799476028, labels=[1 1 4 4 4 4 1 1 4 4 4 4 4 4 4 4 4 4 1 1 4 4 1 1 4 4 1 1 0 0 1 1 1 1 1 1 1
  1 4 4 1 1 3 1 4 0 1 3 3 3 0 2 0 3 2 3 0 0 3 2 3 3 3 3 3 0 3 3 0 3 1 0 3 3
  3 3 3 3 3 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0 3 3 3 3 3 3 3 3 0 0 0 3 3 0
  0 0 3], medoids=[ 71  33  51 101  12], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=4621.892078813149, labels=[3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 0 3 3 3 3 3 3 3
  3 3 3 3 3 2 3 3 0 0 2 2 2 0 1 0 2 1 2 0 2 2 1 2 2 2 2 2 0 2 2 0 2 2 0 2 2
  2 2 2 2 2 0 0 2 2 2 2 3 2 2 2 2 2 2 2 2 0 0 0 2 2 2 2 2 2 2 2 0 0 0 2 2 0
  0 0 2], medoids=[71 51 91 12], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=4862.346682582, labels=[2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 0 2 2 2 2 2 2 2
  2 2 2 2 2 1 2 2 0 0 1 1 1 0 0 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1
  1 1 1 1 1 0 0 1 1 1 1 2 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0
  0 0 1], medoids=[71 91 12], n_iter=5, n_swaps=4))

In [27]:
#Below will take the above data and use it to make the alluvial (sankey) plot

In [28]:
#debug print to make sure I can call the cluster labels from the KMedoidsResult object
hunPamk4.labels

array([0, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 3, 3,
       3, 0, 0, 3, 3, 3, 0, 2, 0, 3, 2, 3, 0, 0, 0, 2, 0, 0, 3, 3, 0, 0,
       3, 3, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0,
       0, 0, 0, 0], dtype=uint64)

In [29]:
# Defining cluster labels for each dataset
labels_ten = tenPamk4.labels
labels_hun = hunPamk4.labels
labels_thous = thousPamk4.labels
labels_ten_thous = tenThousPamk4.labels
labels_five_thous = fiveThousPamk4.labels